In [1]:
import os

In [2]:
%pwd

'C:\\Users\\hp\\Desktop\\Sub\\Text_Summarizer_NLP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\hp\\Desktop\\Sub\\Text_Summarizer_NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path
    

In [30]:
from src.Text_Summarizer_NLP import *

In [27]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [34]:
from src.Text_Summarizer_NLP.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'Text_Summarizer_NLP.logging'